In [1]:
import tensorflow as tf
import keras
import pandas as pd
import numpy
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer
import socket, struct
from sklearn.preprocessing import OneHotEncoder, normalize, MinMaxScaler
from sklearn.model_selection import KFold
from keras.layers import LSTM, Dense
from keras.models import Sequential
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import chi2
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
import time
#import keras_metrics as km
from keras.layers import merge, Input, Dropout, Concatenate,Activation
from keras.models import Model
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from keras.callbacks import Callback

class SkMetrics(Callback):
    def __init__(self,batchsize):
        self.batchsize = batchsize
    def on_train_begin(self, logs={}):
        self.confusion = []
        self.precision = []
        self.recall = []
        self.f1s = []
        self.kappa = []        

    def on_epoch_end(self, epoch, logs={}):
        score = numpy.asarray(self.model.predict(self.validation_data[0],batch_size = self.batchsize))
        predict = numpy.round(numpy.asarray(self.model.predict(self.validation_data[0],batch_size = self.batchsize)))
        targ = self.validation_data[1]
        targ = targ.flatten()
        predict = predict.flatten()
        self.confusion.append(confusion_matrix(targ, predict, labels=[0,1]))
        self.precision.append(precision_score(targ, predict))
        self.recall.append(recall_score(targ, predict))
        self.f1s.append(f1_score(targ, predict))
        self.kappa.append(cohen_kappa_score(targ, predict))

def getModel(neurons, layers, inputshape, batchsize):
    model = Sequential()
    model.add(LSTM(neurons, batch_input_shape = (batchsize, inputshape[0],inputshape[1]), return_sequences=True, stateful=False))
    for i in range(layers):
        model.add(LSTM(neurons, return_sequences=False, stateful=False))
    model.add(Dense(1,activation = 'sigmoid'))
    model.compile(optimizer="adam", 
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

Using TensorFlow backend.


In [2]:
data = pd.read_csv('./UNSW-NB15_1.csv', header=None)
data1a = numpy.asarray(data)
data1a = numpy.delete(data1a, 47, 1)
data2 = pd.read_csv('./UNSW-NB15_2.csv', header=None)
data2a = numpy.asarray(data2)
data2a = numpy.delete(data2a, 47, 1)
data3 = pd.read_csv('./UNSW-NB15_3.csv', header=None)
data3a = numpy.asarray(data3)
data3a = numpy.delete(data3a, 47, 1)
data = []
temp = len(data1a) + len(data2a) + len(data3a)
data.append(data1a)
data.append(data2a)
data.append(data3a)

data = numpy.reshape(numpy.asarray(data),(temp, 48))


/home/vanet/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/vanet/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def iptoint(ip):
    return struct.unpack("!I",(socket.inet_aton(ip)))[0] 

def parse(data):
    protocols = dict()
    protocolsNum = 1
    services  = dict()
    servicesNum = 1
    states = dict()
    statesNum = 1
    numMalicious = 0
    numNormal = 0
    x = []
    y = []
    for dataRaw in data:
        dataRaw[0] = iptoint(dataRaw[0])
        dataRaw[2] = iptoint(dataRaw[2])
        #Source IP
        if (type(dataRaw[39]) == str):
            if (dataRaw[39] !=  " "):
                dataRaw[39] = int(dataRaw[39],0)
            else:
                dataRaw[39] = 0
        if (type(dataRaw[1]) == str):
            if (dataRaw[1] != "-"):
                dataRaw[1] = int(dataRaw[1],0)
            else:
                dataRaw[1] = 0
        #Dest IP
        if (type(dataRaw[3]) == str):
            if (dataRaw[3] != "-"):
                dataRaw[3] = int(dataRaw[3],0)
            else:
                dataRaw[3] = 0
        if (dataRaw[4] in protocols):
            dataRaw[4] = protocols.get(dataRaw[4])
        else:
            protocols.update({dataRaw[4]:protocolsNum})
            dataRaw[4] = protocolsNum
            protocolsNum += 1

        if (type(dataRaw[4])!= int):
            print (dataRaw[4])
 
        if (dataRaw[5] in states):
            dataRaw[5] = states.get(dataRaw[5])
        else:
            states.update({dataRaw[5]:statesNum})
            dataRaw[5] = statesNum
            statesNum += 1
        if (type(dataRaw[5])!=int):
            print (dataRaw[5])
        if (dataRaw[13] in services):
            dataRaw[13] = services.get(dataRaw[13])
        else:
            services.update({dataRaw[13]:servicesNum})
            dataRaw[13] = servicesNum
            servicesNum += 1
        if (type(dataRaw[13])!=int):
            print (dataRaw[13])
        if (dataRaw[47] == 1):
            numMalicious = numMalicious + 1
            y.append(1)
        else:
            numNormal = numNormal + 1
            y.append(0)
        i = 0
        for data in dataRaw:
            if (type(data) == str):
                print(dataRaw,data,i)
            i = i+1
        x.append(dataRaw)
    print("normal = ",numNormal)
    print("Malicious = ", numMalicious)
    print(protocols)
    print(states)
    print(services)
    return [x, y]


def optimizeParameters(data, labels, scorefunc):
    # Выбираем k лучших признаков
    selecter = SelectKBest(score_func=scorefunc, k=10)
    selecter.fit(data, labels)
    
    # Какие именно выбрали
    params = []
    index = []
    params = selecter.get_support()
    count = 0
    
    for param in params :
        if param == True :
            index.append(count)
        count += 1
    

    index_file = open("index.txt" , "w")
    for i in index : 
        index_file.write(str(i) + " ")
    index_file.close()

    return selecter.transform(data)

def drawGraphic(dataset, labels, normalCount, malwareCount):

    parametersCount = len(dataset[0])

    normalCountList = [0] * parametersCount
    malwareCountList = [0] * parametersCount

    for i in range(0, len(dataset)):
        if(i < normalCount):
            for j in range(0, parametersCount):
                normalCountList[j] += dataset[i][j]
        else:
            for j in range(0, parametersCount):
                malwareCountList[j] += dataset[i][j]

    for i in range(0, parametersCount):
        normalCountList[i] = normalCountList[i]
        malwareCountList[i] = malwareCountList[i] 

    ts = pd.DataFrame({'normal': normalCountList,
                       'malware': malwareCountList})
    ts.plot()
    plt.show()
    return


    
def ROC(ROC_MODEL , ROC_X, ROC_Y):
    probs = ROC_MODEL.predict(ROC_X)
    ROC_Y = ROC_Y.flatten()
    probs = probs.flatten()
    FPr, TPr, threshold = metrics.roc_curve(ROC_Y, probs)
    roc_auc = metrics.auc(FPr, TPr)
    lw = 2
    
    plt.figure()
    plt.plot(FPr, TPr, lw = lw , label='Logistic Regression (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.savefig('Log_ROC')
    plt.show()

In [4]:

[x, y] = parse(data)
min_max_scaler = MinMaxScaler()
newx = min_max_scaler.fit_transform(x)
newx = numpy.delete(numpy.asarray(newx),47,1)
newy = y
print(len(newx))
newx = newx.tolist()
print(newx[0])
print(newy[0])

normal =  1867614
Malicious =  232389
{'udp': 1, 'arp': 2, 'tcp': 3, 'ospf': 4, 'icmp': 5, 'igmp': 6, 'sctp': 7, 'udt': 8, 'sep': 9, 'sun-nd': 10, 'swipe': 11, 'mobile': 12, 'pim': 13, 'rtp': 14, 'ipnip': 15, 'ip': 16, 'ggp': 17, 'st2': 18, 'egp': 19, 'cbt': 20, 'emcon': 21, 'nvp': 22, 'igp': 23, 'xnet': 24, 'argus': 25, 'bbn-rcc': 26, 'chaos': 27, 'pup': 28, 'hmp': 29, 'mux': 30, 'dcn': 31, 'prm': 32, 'trunk-1': 33, 'xns-idp': 34, 'trunk-2': 35, 'leaf-1': 36, 'leaf-2': 37, 'irtp': 38, 'rdp': 39, 'iso-tp4': 40, 'netblt': 41, 'mfe-nsp': 42, 'merit-inp': 43, '3pc': 44, 'xtp': 45, 'idpr': 46, 'tp++': 47, 'ddp': 48, 'idpr-cmtp': 49, 'ipv6': 50, 'il': 51, 'idrp': 52, 'ipv6-frag': 53, 'sdrp': 54, 'ipv6-route': 55, 'gre': 56, 'rsvp': 57, 'mhrp': 58, 'bna': 59, 'esp': 60, 'i-nlsp': 61, 'narp': 62, 'ipv6-no': 63, 'tlsp': 64, 'skip': 65, 'ipv6-opts': 66, 'any': 67, 'cftp': 68, 'sat-expak': 69, 'kryptolan': 70, 'rvd': 71, 'ippc': 72, 'sat-mon': 73, 'ipcv': 74, 'visa': 75, 'cpnx': 76, 'cphb': 77, 

In [5]:

while(len(newx)%100000 != 0):
    newx.pop()
    newy.pop()
print(len(newx))

2100000


In [6]:
print (newx[0])
print (len(newx[0]))
print (len(newx))

def construct_data_for_package_memory_len(package_len, x, y):
    x_result = []
    y_result = []
    for j in range (len(x) // package_len - 1):
        for i in range (package_len):
            temp_arr = []
            for k in range (package_len):
                temp_arr.append(x[j * package_len + i + k])
            x_result.append(temp_arr)
            y_result.append(y[j * package_len + i + package_len - 1])
    while len(x_result) % 10000 != 0:
        x_result.pop()
        y_result.pop()
    for i in range(len(x_result)):
        for j in range(len(x_result[i])):
            for k in range(len(x_result[i][j])):
                if numpy.isnan(x_result[i][j][k]):
                    x_result[i][j][k] = 1
    return [x_result, y_result]


[0.2711794028719652, 0.021210040436408027, 0.6524071462129994, 9.83321850267745e-08, 0.0, 0.0, 1.2006868117486436e-07, 9.194906523326434e-06, 1.1188787524993125e-05, 0.12156862745098039, 0.11417322834645668, 0.0, 0.0, 0.0, 8.357947830722337e-05, 0.0048290753567918655, 0.00018786398647379298, 0.0001815211472136504, 0.0, 0.0, 0.0, 0.0, 0.043882978723404256, 0.05466666666666666, 0.0, 0.0, 0.0, 0.0, 1.593020328982675e-05, 0.0, 2.8328649663255683e-07, 2.185413140586619e-07, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.030303030303030304, 0.09090909090909091, 0.0, 0.030303030303030304, 0.0, 0.0, 0.0]
47
2100000


In [7]:
def CrossValidation(lenOfBlock , x, y, neurons, layers, inputshape, epochsIn, batchsize):
    cv = KFold(n_splits = lenOfBlock, random_state = None, shuffle = True)
        
    TN = 0
    FP = 0
    FN = 0
    TP = 0
    F1 = 0
    Educate = 0.0
    Test = 0.0
    print("Neurons: ", neurons)
    print("Layers: ", layers)
    numblock=0
    x = numpy.reshape(x,(5,len(x)//5,inputshape[0],47))
    y = numpy.reshape(y,(5,len(y)//5))
    
    
    for train_index, test_index in cv.split(x[0]):
        clf = getModel(neurons, layers, inputshape, batchsize)
        print("TRAIN:", train_index, "TEST:", test_index)
        print("Block: ", numblock)
        numblock +=1
        start_time = time.time()
        skmetrics = SkMetrics(batchsize)
        for i in range (epochsIn):
            for d in range (5):
                history = clf.fit(x[d][train_index], y[d][train_index], callbacks = [skmetrics], epochs = 1, validation_data = (x[d][test_index], y[d][test_index]), batch_size = batchsize)        
        for k in skmetrics.confusion:
            print(k)
        education_time = time.time() - start_time
        try:
            # Тестирование
            start_time = time.time()
            proba = []
            for i in range(5):
                proba.append(numpy.round(numpy.asarray(clf.predict(x[i][test_index], batch_size = batchsize))))
                
            test_time = time.time() - start_time
            # Добавляем значения
            Educate += education_time
            Test += test_time
            y1 = []
            for i in range(5):
                y1.append(y[i][test_index].flatten())
                cn1 = confusion_matrix(y1[i], proba[i], labels=[0,1])
                tn1, fp1, fn1, tp1 = cn1.ravel()
                TN += tn1 
                FP += fp1 
                FN += fn1
                TP += tp1 
                F1 += (f1_score(y1[i], proba[i] , average='binary'))
                print(cn1)
        except ZeroDivisionError:
            print("ZeroDivisionError")
       
        
    summ = TN + FP + FN + TP
    
    print("\n\n\n")
    print("Block len " + str(lenOfBlock))
    print("True positive " + str(TP / summ), TP)
    print("True negative " + str(TN / summ), TN)
    print("False positive " + str(FP / summ), FP)
    print("False negative " + str(FN / summ), FN)
   
    print("Accuracy " + str((TP + TN)/(summ)))
    print("Precision " + str(TP / (TP + FP))) # Precision
    print("Recall " + str(TP / (TP + FN))) # Recall

    print("F1 score " + str(F1 / lenOfBlock)) # F1 score
    print("Educate time " + str(Educate / lenOfBlock)) # Educate time
    print("Test time " + str(Test / lenOfBlock)) # Test time

    print("\n\n\n")    

In [8]:
#[x, y] = construct_data_for_package_memory_len(5, newx, newy)
#x1 = numpy.array(x)
#y1 = numpy.array(y)
#CrossValidation(5 , x1, y1, 100, 1, (5,47), 2, 200)

In [9]:
[x, y] = construct_data_for_package_memory_len(40, newx, newy)
x1 = numpy.array(x)
y1 = numpy.array(y)
CrossValidation(5 , x1, y1, 100, 1, (40,47), 2, 200)


Neurons:  100
Layers:  1
TRAIN: [     3      5      6 ... 417996 417997 417998] TEST: [     0      1      2 ... 417988 417992 417999]
Block:  0
Train on 334400 samples, validate on 83600 samples
Epoch 1/1
334400/334400 [==============================] - 496s 1ms/step - loss: 0.0272 - acc: 0.9903 - val_loss: 0.0112 - val_acc: 0.9956
Train on 334400 samples, validate on 83600 samples
Epoch 1/1
334400/334400 [==============================] - 487s 1ms/step - loss: 5.7463e-06 - acc: 1.0000 - val_loss: 9.6498e-07 - val_acc: 1.0000


/home/vanet/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/vanet/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/home/vanet/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/vanet/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/home/vanet/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:576: 

Train on 334400 samples, validate on 83600 samples
Epoch 1/1
334400/334400 [==============================] - 483s 1ms/step - loss: 0.0140 - acc: 0.9944 - val_loss: 0.0092 - val_acc: 0.9963
Train on 334400 samples, validate on 83600 samples
Epoch 1/1
334400/334400 [==============================] - 471s 1ms/step - loss: 0.0218 - acc: 0.9909 - val_loss: 0.0180 - val_acc: 0.9925
Train on 334400 samples, validate on 83600 samples
Epoch 1/1
334400/334400 [==============================] - 412s 1ms/step - loss: 0.0185 - acc: 0.9921 - val_loss: 0.0182 - val_acc: 0.9921
Train on 334400 samples, validate on 83600 samples
Epoch 1/1
334400/334400 [==============================] - 405s 1ms/step - loss: 0.0088 - acc: 0.9965 - val_loss: 0.0083 - val_acc: 0.9967
Train on 334400 samples, validate on 83600 samples
Epoch 1/1
334400/334400 [==============================] - 402s 1ms/step - loss: 1.1348e-05 - acc: 1.0000 - val_loss: 1.5881e-06 - val_acc: 1.0000
Train on 334400 samples, validate on 83600

[[78999   199]
 [  129  4273]]
[[83599     1]
 [    0     0]]
[[77467   183]
 [   80  5870]]
[[65786   370]
 [  223 17221]]
[[64697   355]
 [  161 18387]]
TRAIN: [     0      1      2 ... 417995 417998 417999] TEST: [     3      6      7 ... 417994 417996 417997]
Block:  4
Train on 334400 samples, validate on 83600 samples
Epoch 1/1
334400/334400 [==============================] - 456s 1ms/step - loss: 0.0278 - acc: 0.9905 - val_loss: 0.0109 - val_acc: 0.9957
Train on 334400 samples, validate on 83600 samples
Epoch 1/1
334400/334400 [==============================] - 410s 1ms/step - loss: 9.2887e-06 - acc: 1.0000 - val_loss: 1.6423e-06 - val_acc: 1.0000
Train on 334400 samples, validate on 83600 samples
Epoch 1/1
334400/334400 [==============================] - 412s 1ms/step - loss: 0.0130 - acc: 0.9948 - val_loss: 0.0098 - val_acc: 0.9960
Train on 334400 samples, validate on 83600 samples
Epoch 1/1
334400/334400 [==============================] - 411s 1ms/step - loss: 0.0214 - acc: 0.

In [ ]:
[x, y] = construct_data_for_package_memory_len(20, newx, newy)
x1 = numpy.array(x)
y1 = numpy.array(y)
CrossValidation(5 , x1, y1, 100, 1, (20,47), 2, 200)

In [ ]:
[x, y] = construct_data_for_package_memory_len(2, newx, newy)
x1 = numpy.array(x)
y1 = numpy.array(y)
CrossValidation(5 , x1, y1, 100, 1, (2,47), 2, 200)

[x, y] = construct_data_for_package_memory_len(4, newx, newy)
x1 = numpy.array(x)
y1 = numpy.array(y)
CrossValidation(5 , x1, y1, 100, 1, (4,47), 2, 200)

In [ ]:
[x, y] = construct_data_for_package_memory_len(7, newx, newy)
x1 = numpy.array(x)
y1 = numpy.array(y)
CrossValidation(5 , x1, y1, 100, 1, (7,47), 2, 200)

[x, y] = construct_data_for_package_memory_len(3, newx, newy)
x1 = numpy.array(x)
y1 = numpy.array(y)
CrossValidation(5 , x1, y1, 100, 1, (3,47), 2, 200)